# Project test

In this assignment, we are going to improve our codes in PattRecClasses and implement forward algorithm inside MarkovChain code as well as functions such as logprob and prob in Guassian in order to generate proper input values for forward algorithm.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats
from matplotlib import pyplot as plt

from CharacterFeatureExtractor import featureExtractor
from DrawCharacter import DrawCharacter
from PattRecClasses import HMM_TA 
from hmm_gen import hmm_gen

In [2]:
'''
Multivariate Gaussian Distribution Class
'''
class multigaussD:
    mean = np.array([0])
    cov = np.array([[0]])
    def __init__(self, mu, C):
        if C.shape[0] is not C.shape[1]:
            print("error, non-square covariance matrix supplied")
            return
        if mu.shape[0] is not C.shape[0]:
            print("error, mismatched mean vector and covariance matrix dimensions")
            return
        self.mean = mu
        if np.where(np.diag(C)==0)[0].shape[0] != 0:
            C += np.diagflat(np.ones(C.shape[0])/10000)
        C[np.isnan(C)]=1
        self.cov = C
        return
    def random(self, num):
        return np.random.multivariate_normal(self.mean, self.cov, num)
    def rand(self):
        return np.random.multivariate_normal(self.mean, self.cov, 1)[0]
    def likelihood(self, X):
        p = scipy.stats.multivariate_normal(self.mean, self.cov, 1)
        pd = p.pdf(X)
        return pd
    def loghood(self, X):
        return np.log(self.likelihood(X))
    def getmean(self):
        return self.mean
    def getcov(self):
        return self.cov
    
def prob(x, B):
    T = x.shape[0]
    N = B.shape[0]
    res = np.zeros((T, N))
    for i in range(T):
        for j in range(N):
            res[i,j] = B[j].likelihood(x[i])
    scaled = np.zeros(res.shape)
    for i in range(scaled.shape[0]):
        for j in range(scaled.shape[1]):
            scaled[i, j] = res[i,j]/np.amax(res[i])
    return res, scaled


def logprob(x, B):
    res, scaled = prob(x,B)
    return np.log(res), np.log(scaled)

# Code skeleton

In [3]:
### data prep
db_name = "database_test"
data_features = pd.read_pickle(r'data/' + db_name + '_features.cdb')
data_labels = pd.read_pickle(r'data/' + db_name + '_labels.cdb')

# data_features[k][r] == np.array (ndim, t); K (number of letters) of R samples with Tr individual lengths
# print((data_features[1][1].shape))
print(data_labels)

# train for one character (seen from labels)

hm_learn = [0,0,0,0,0]

# Start by assigning initial HMM parameter values,
# then refine these iteratively

# A (char = 0)
# States = 3
qstar = np.array([1, 0, 0])
Astar = np.array([[0.9, 0.1, 0], [0, 0.9, 0.1], [0, 0, 1]])
meansstar = np.array([[10, 60], [10, -70], [40,0]])
covsstar = np.array([[[1, 1], [1, 1]],
                     [[1, 1], [1, 1]], [[1,1],[1,1]]])

print(covsstar)

Bstar = np.array([multigaussD(meansstar[0], covsstar[0]),
                  multigaussD(meansstar[1], covsstar[1]), 
                  multigaussD(meansstar[2], covsstar[2])])

hm_learn[0] = HMM_TA.HMM(qstar, Astar, Bstar)

# C (char = 1)
# States = 5 ...

# X (char = 4)
# States = 2
qstar = np.array([1, 0])
Astar = np.array([[0.9, 0.1], [0, 1]])
meansstar = np.array([[10, -60], [45, 70]])
covsstar = np.array([[[1, 1], [1, 1]],
                     [[1, 1], [1, 1]]])
Bstar = np.array([multigaussD(meansstar[0], covsstar[0]),
                  multigaussD(meansstar[1], covsstar[1])])

print(meansstar[0])


hm_learn[4] = HMM_TA.HMM(qstar, Astar, Bstar)

hm_learn[1] = hm_learn[0]
hm_learn[2] = hm_learn[0]
hm_learn[3] = hm_learn[0]

['A', 'C', 'K', 'P', 'X']
[[[1 1]
  [1 1]]

 [[1 1]
  [1 1]]

 [[1 1]
  [1 1]]]
[ 10 -60]


In [4]:
hm_learn = hmm_gen(data_features, 10)

for char in range(len(data_features)):
    print("\n")
    print("------------ CHARACTER", data_labels[char], "------------")
    # Data preprocessing 
    obs = data_features[char]
    # oos_obs = data_features[char-1]
    
    # obsTA = np.array([ hm_learn.rand(100)[0] for _ in range(10) ])
    # print(type(obsTA))
    # print(obsTA[1].shape) == (100,2)
    # Our data has format (2,15) ! Transpose all datapoints
    for i in range(len(obs)):
        obs[i] = np.transpose(obs[i])
    
    #for i in range(len(oos_obs)):
    #    oos_obs[i] = np.transpose(oos_obs[i])

    # Data information
    """
    print(len(obs))
    print(obs[len(obs) - 1].shape)
    print(type(obs))
    print(obs[1])
    """

    # Divide data into training and testing
    train_obs = obs[0:3]
    test_obs = obs[4]

    # Training
    print("Running the Baum Welch Algorithm...")
    hm_learn[char].baum_welch(train_obs, 20, prin=1, uselog=False)

    
    # Testing on out of sample and test obs
    
    #a, c = hm_learn[char].alphahat(oos_obs[2])
    #print("Prob oos", c)
    
    a, c = hm_learn[char].alphahat(test_obs)
    print("Prob is", c)

------------ CHARACTER 0 ------------
Number of states 4
qstar [1. 0. 0. 0.]
Astar [[0.9 0.1 0.  0. ]
 [0.  0.9 0.1 0. ]
 [0.  0.  0.9 0.1]
 [0.  0.  0.  1. ]]
Bstar [[  7.81078884  50.19884682]
 [  9.19274596  -5.79302865]
 [ 40.06537137 -32.73080836]
 [ 41.90309371 -16.36319523]]


------------ CHARACTER 1 ------------
Number of states 6
qstar [1. 0. 0. 0. 0. 0.]
Astar [[0.9 0.1 0.  0.  0.  0. ]
 [0.  0.9 0.1 0.  0.  0. ]
 [0.  0.  0.9 0.1 0.  0. ]
 [0.  0.  0.  0.9 0.1 0. ]
 [0.  0.  0.  0.  0.9 0.1]
 [0.  0.  0.  0.  0.  1. ]]
Bstar [[  9.69096999  -2.11022837]
 [  9.43982274  32.00855209]
 [  9.37609837  30.00316888]
 [  7.81609129 -50.19126002]
 [  9.85469941 -23.95932009]
 [  9.55847199  -6.02222356]]


------------ CHARACTER 2 ------------
Number of states 4
qstar [1. 0. 0. 0.]
Astar [[0.9 0.1 0.  0. ]
 [0.  0.9 0.1 0. ]
 [0.  0.  0.9 0.1]
 [0.  0.  0.  1. ]]
Bstar [[  9.74360566  34.18011984]
 [  9.53145007   3.17394648]
 [ 37.11074763  24.34065709]
 [ 37.43876266 -35.58059642

/home/jonas/Local/Pattern-Recognition-Project/PattRecClasses/HMM_TA.py:96: RuntimeWarning: overflow encountered in true_divide
  beta[t] = temp / cs[t]
/home/jonas/Local/Pattern-Recognition-Project/PattRecClasses/HMM_TA.py:95: RuntimeWarning: invalid value encountered in double_scalars
  temp[i] += A[i, j] * scaled[t + 1, j] * beta[t + 1, j]
/home/jonas/Local/Pattern-Recognition-Project/PattRecClasses/HMM_TA.py:145: RuntimeWarning: invalid value encountered in multiply
  temp += [alphahats[i][t] * betahats[i][t] * cs[i][t]]
/home/jonas/Local/Pattern-Recognition-Project/PattRecClasses/HMM_TA.py:189: RuntimeWarning: invalid value encountered in double_scalars
  xi[t, j, k] = alphahats[i][t][j] * self.A[j, k] * scaled[t + 1][k] * betahats[i][t + 1][k]
/home/jonas/Local/Pattern-Recognition-Project/PattRecClasses/HMM_TA.py:323: RuntimeWarning: invalid value encountered in double_scalars
  scaled[i, j] = res[i, j] / np.amax(res[i])


Estimated a:
[nan nan nan nan nan nan]

Estimated A:
[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]

Estimated means:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

Estimated covariances:
[[[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]]
Prob is [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan]


------------ CHARACTER K ------------
Running the Baum Welch Algorithm...


/home/jonas/Local/Pattern-Recognition-Project/PattRecClasses/HMM_TA.py:61: RuntimeWarning: invalid value encountered in true_divide
  alpha[t, :] = temp / c[t]


Estimated a:
[nan nan nan nan]

Estimated A:
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]

Estimated means:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

Estimated covariances:
[[[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]]
Prob is [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]


------------ CHARACTER P ------------
Running the Baum Welch Algorithm...
Estimated a:
[nan nan nan]

Estimated A:
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]

Estimated means:
[[0. 0.]
 [0. 0.]
 [0. 0.]]

Estimated covariances:
[[[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]

 [[0.0001 0.    ]
  [0.     0.0001]]]
Prob is [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan na